# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition.

# 1) Importing data and libraries

In [15]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from sklearn.preprocessing import MinMaxScaler
from sentence_transformers import SentenceTransformer, CrossEncoder, util

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [16]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = '../subtask4b_collection_data.pkl' #MODIFY PATH

In [17]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [18]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [19]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [20]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4/subtask_4b?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_TRAIN_DATA = '../subtask4b_query_tweets_train.tsv' #MODIFY PATH
PATH_QUERY_DEV_DATA = '../subtask4b_query_tweets_dev.tsv' #MODIFY PATH
PATH_QUERY_TEST_DATA = '../subtask4b_query_tweets_test.tsv' #MODIFY PATH

In [21]:
df_query_train = pd.read_csv(PATH_QUERY_TRAIN_DATA, sep = '\t')
df_query_dev = pd.read_csv(PATH_QUERY_DEV_DATA, sep = '\t')

In [22]:
df_query_train.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [23]:
df_query_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [24]:
df_query_dev.head()

,post_id,tweet_text,cord_uid
0,16,covid recovery: this study from the usa reveal...,3qvh482o
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu
2,73,I recall early on reading that researchers who...,sts48u9i
3,93,You know you're credible when NIH website has ...,3sr2exq9
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy


In [25]:
df_query_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


# 2) Importing Baselinemodels

## 2.1) Bi-Encoder

In [27]:
from sentence_transformers import SentenceTransformer, util
import torch
from tqdm import tqdm

bi_encoder = SentenceTransformer('all-MiniLM-L6-v2')

corpus = df_collection[['title', 'abstract']].apply(
    lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection['cord_uid'].tolist()

corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\mayer\anaconda3\envs\AIR_project_25\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mayer\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/242 [00:00<?, ?it/s]

In [28]:
def get_top_cord_uids_bi_encoder(query, top_k=20):
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    
    top_cord_uids = [cord_uids[idx] for idx in top_results.indices]
    return top_cord_uids


In [29]:
tqdm.pandas()
df_query_train['bi_topk'] = df_query_train['tweet_text'].progress_apply(lambda x: get_top_cord_uids_bi_encoder(x))
df_query_dev['bi_topk'] = df_query_dev['tweet_text'].progress_apply(lambda x: get_top_cord_uids_bi_encoder(x))


100%|██████████| 1400/1400 [00:56<00:00, 24.57it/s]


## 2.1.1) Evaluating the baseline bi-encoder

In [30]:
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [31]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bi_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bi_topk')


In [32]:
results_train

{1: np.float64(0.39313778884307166),
 5: np.float64(0.46201016623875096),
 10: np.float64(0.47126021471116003)}

In [33]:
results_dev

{1: np.float64(0.4157142857142857),
 5: np.float64(0.48966666666666664),
 10: np.float64(0.4993826530612245)}

# 2.2) Importing BM-25 baseline
The following code runs a BM25 baseline.


In [35]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

**Note:** After several attempts, we noticed that the top_k with k=20 receives the best results

In [14]:
def get_top_cord_uids(query):
  text2bm25top = {}
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:20]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk


In [36]:
# Retrieve topk candidates using the BM25 model
tqdm.pandas()
df_query_train['bm25_topk'] = df_query_train['tweet_text'].progress_apply(lambda x: get_top_cord_uids(x))
df_query_dev['bm25_topk'] = df_query_dev['tweet_text'].progress_apply(lambda x: get_top_cord_uids(x))

100%|██████████| 1400/1400 [03:09<00:00,  7.37it/s]


In [37]:
df_query_train['bm25_topk'][1]

['2cwvga0k',
 '33znyrn8',
 'st3fyb64',
 '74dw6emg',
 'b68c8qwm',
 '7xt894vr',
 'q1mq5f2t',
 'f3eogz0n',
 'xua4f77e',
 '4vkkaqhz',
 'gg5c8v7d',
 'yj652emy',
 'eoumpu50',
 'moe0bi1g',
 '29z4q4fs',
 'pva0xii7',
 'ksveiiid',
 'usi65ft4',
 'fm8koqjd',
 'zpqgz95r']

## 2.2.1) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [30]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [31]:
results_train = get_performance_mrr(df_query_train, 'cord_uid', 'bm25_topk')
results_dev = get_performance_mrr(df_query_dev, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(f"Results on the train set: {results_train}")
print(f"Results on the dev set: {results_dev}")

Results on the train set: {1: np.float64(0.5081303975725512), 5: np.float64(0.5509777224513084), 10: np.float64(0.5559614579512658)}
Results on the dev set: {1: np.float64(0.505), 5: np.float64(0.5519166666666666), 10: np.float64(0.5573010204081632)}


# 2.3) Exporting results to prepare the submission on Codalab

In [32]:
df_query_dev['preds'] = df_query_dev['bm25_topk'].apply(lambda x: x[:5])

In [57]:
df_query_dev[['post_id', 'preds']].to_csv('predictions_baseline.tsv', index=None, sep='\t')

In [34]:
df_query_dev.head()

,post_id,tweet_text,cord_uid,hybrid_cross_encoder_topk,cross_encoder_topk,hybrid_topk,bm25_topk,in_topx,preds
0,16,covid recovery: this study from the usa reveal...,3qvh482o,None,None,None,"[25aj8rj5, gatxuwz7, 59up4v56, styavbvi, 6sy80...",0.000000,"[25aj8rj5, gatxuwz7, 59up4v56, styavbvi, 6sy80..."
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,None,None,None,"[r58aohnu, p0kg6dyz, 9dlaaye8, iu1d9i57, d06np...",1.000000,"[r58aohnu, p0kg6dyz, 9dlaaye8, iu1d9i57, d06np..."
2,73,I recall early on reading that researchers who...,sts48u9i,None,None,None,"[tz2shoso, o877uul1, m1sf159a, sgo76prc, gruir...",0.166667,"[tz2shoso, o877uul1, m1sf159a, sgo76prc, gruir..."
3,93,You know you're credible when NIH website has ...,3sr2exq9,None,None,None,"[3sr2exq9, hgpiig0g, sv48gjkk, k0f4cwig, ihgxt...",1.000000,"[3sr2exq9, hgpiig0g, sv48gjkk, k0f4cwig, ihgxt..."
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,None,None,None,"[lzddnb8j, ouvq2wpq, sxx3yid9, vabb2f26, y9fqa...",0.125000,"[lzddnb8j, ouvq2wpq, sxx3yid9, vabb2f26, y9fqa..."


In [35]:
df_query_train.head()

,post_id,tweet_text,cord_uid,hybrid_topk,bm25_topk,in_topx
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,None,"[htlvpvz5, h7hj64q5, rwgqkow3, dbgtslc8, am11y...",1.000000
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,None,"[2cwvga0k, 33znyrn8, st3fyb64, 74dw6emg, b68c8...",0.000000
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,None,"[8hkxbxz9, w98847ai, jtwb17u8, iy1enazk, pg0l9...",0.333333
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,None,"[32ua8wb6, zxe95qy9, 30pl5tx3, l4y7v729, b97ac...",0.000000
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,None,"[tiqksd69, k7smwz6w, b0dzhsrh, yc7cvbii, iiafu...",1.000000


## 3) Re-ranking approach

In [36]:
# Initialize Cross-Encoder once
cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L2-v2', max_length=512)

def rank_chunks_with_cross_encoder(query, retrieved_chunks, top_k=5):
    """
    Rank the retrieved chunks based on their relevance to the query using a Cross-Encoder.
    """
    query_chunk_pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(query_chunk_pairs)
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return ranked_chunks[:top_k]

# Initialize new column
df_query_dev["cross_encoder_topk"] = None

# Loop over the rows to fill in reranked results
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    topk_ids = row["bm25_topk"]

    chunk_to_uid = {}
    for uid in topk_ids:
        paper_row = df_collection.loc[df_collection["cord_uid"] == uid]
        if paper_row.empty:
            continue

        title = paper_row["title"].values[0] or ""
        abstract = paper_row["abstract"].values[0] or ""
        full_text = f"{title.strip()}\n{abstract.strip()}"
        chunk_to_uid[full_text] = uid

    reranked_chunks = rank_chunks_with_cross_encoder(query_text, list(chunk_to_uid.keys()))
    reranked_uids = [chunk_to_uid[chunk] for chunk, _ in reranked_chunks]

    # Assign directly to the dataframe
    df_query_dev.at[idx, "cross_encoder_topk"] = reranked_uids

# Preview result
print(df_query_dev[["cord_uid", "cross_encoder_topk"]].head())



100%|██████████| 1400/1400 [02:49<00:00,  8.25it/s]

   cord_uid                                 cross_encoder_topk
0  3qvh482o  [o4vvlmr4, hg3xpej0, 25aj8rj5, 59up4v56, bqn29...
1  r58aohnu  [r58aohnu, tu1vevx9, yrowv62k, mcmqx7og, zd8c1...
2  sts48u9i  [osct0lj6, o47v5vgw, sts48u9i, gruir7aw, o877u...
3  3sr2exq9  [kdegnr6i, 3sr2exq9, k0f4cwig, sv48gjkk, z795y...
4  ybwwmyqy  [ybwwmyqy, vabb2f26, ouvq2wpq, sxx3yid9, lzddn...


In [37]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'cross_encoder_topk')
print(f"Results on the reranked set: {results_reranked}")

Results on the reranked set: {1: np.float64(0.55), 5: np.float64(0.5952857142857143), 10: np.float64(0.5952857142857143)}


# 4) New approach

## 4.1) Use hybrid approach as baseline

In [39]:
corpus = df_collection.progress_apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection['cord_uid'].tolist()

# Encode corpus once
bi_encoder = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True)

def hybrid_retrieve(query, bm25, corpus_embeddings, cord_uids, top_k=20):
    # Step 1: Get BM25 scores
    bm25_scores = bm25.get_scores(query.split())  # shape: (N,)

    # Step 2: Get dense (bi-encoder) similarity scores
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    dense_scores = util.cos_sim(query_embedding, corpus_embeddings)[0].cpu().numpy()  # shape: (N,)

    # Step 3: Normalize both score types
    scaler = MinMaxScaler()
    combined_scores = scaler.fit_transform(np.vstack([bm25_scores, dense_scores]).T)  # shape: (N, 2)
    hybrid_scores = combined_scores.sum(axis=1)  # shape: (N,)

    # Step 4: Get top-k documents based on hybrid score
    top_indices = np.argsort(-hybrid_scores)[:top_k]
    top_docs = [cord_uids[i] for i in top_indices]

    return top_docs

100%|██████████| 7718/7718 [00:00<00:00, 67645.96it/s]


In [40]:
df_query_dev["hybrid_topk"] = None

# Apply hybrid retrieval to each row
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    top_docs = hybrid_retrieve(query_text, bm25, corpus_embeddings, cord_uids, top_k=20)
    df_query_dev.at[idx, "hybrid_topk"] = top_docs

100%|██████████| 1400/1400 [05:42<00:00,  4.09it/s]


In [41]:
df_query_dev["hybrid_topk"]

0       [25aj8rj5, styavbvi, jrqlhjsm, hg3xpej0, o4vvl...
1       [r58aohnu, yrowv62k, d06npvro, mcmqx7og, mz1bo...
2       [sts48u9i, 8m59whf9, gruir7aw, qkg8fwbp, osct0...
3       [3sr2exq9, k0f4cwig, z795y51f, sv48gjkk, 4fzzo...
4       [ybwwmyqy, lzddnb8j, vabb2f26, ouvq2wpq, sxx3y...
                              ...                        
1395    [0gn3b98n, 0e97tdez, soa71kwx, d8x3b9a3, n5sei...
1396    [92z77upt, s1qnjgev, 763v4duh, 64ugdvfe, q10pv...
1397    [qn6wawxk, 059oj76m, rtus1qyw, wrsk5vh9, t8jli...
1398    [47p4g80o, 2tax8ajw, rfv0omd6, unl3n0tn, wt6az...
1399    [nih4l4ok, w4dyadpa, dfhxcqr8, p82l6d9a, msut5...
Name: hybrid_topk, Length: 1400, dtype: object

In [42]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'hybrid_topk')
print(f"Results on the hybrid set: {results_reranked}")

Results on the hybrid set: {1: np.float64(0.57), 5: np.float64(0.6207261904761905), 10: np.float64(0.6278222789115646)}


## 4.2) Rerank hybrid approach
Here, we make use of the ms-marco model again to re-rank and find out if we can improve our results by the hybrid approach

In [43]:
# Initialize Cross-Encoder once

cross_encoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L2-v2', max_length=512)
#cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L6-v2', max_length=512)

def rank_chunks_with_cross_encoder(query, retrieved_chunks, top_k=5):
    """
    Rank the retrieved chunks based on their relevance to the query using a Cross-Encoder.
    """
    query_chunk_pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(query_chunk_pairs)
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return ranked_chunks[:top_k]

# Initialize new column
df_query_dev["hybrid_cross_encoder_topk"] = None

# Loop over the rows to fill in reranked results
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    topk_ids = row["hybrid_topk"]

    chunk_to_uid = {}
    for uid in topk_ids:
        paper_row = df_collection.loc[df_collection["cord_uid"] == uid]
        if paper_row.empty:
            continue

        title = paper_row["title"].values[0] or ""
        abstract = paper_row["abstract"].values[0] or ""
        full_text = f"{title.strip()}\n{abstract.strip()}"
        chunk_to_uid[full_text] = uid

    reranked_chunks = rank_chunks_with_cross_encoder(query_text, list(chunk_to_uid.keys()))
    reranked_uids = [chunk_to_uid[chunk] for chunk, _ in reranked_chunks]

    # Assign directly to the dataframe
    df_query_dev.at[idx, "hybrid_cross_encoder_topk"] = reranked_uids

# Preview result
print(df_query_dev[["cord_uid", "hybrid_cross_encoder_topk"]].head())

100%|██████████| 1400/1400 [39:08<00:00,  1.68s/it]

   cord_uid                          hybrid_cross_encoder_topk
0  3qvh482o  [nksd3wuw, 59up4v56, 3qvh482o, jrqlhjsm, mb18f...
1  r58aohnu  [r58aohnu, yrowv62k, icgsbelo, wk61uyrt, kiq6x...
2  sts48u9i  [sts48u9i, gruir7aw, o47v5vgw, osct0lj6, u5nxm...
3  3sr2exq9  [kc3nb2pc, k0f4cwig, 3sr2exq9, wbw7gbfv, 9mdf9...
4  ybwwmyqy  [ybwwmyqy, lzddnb8j, vabb2f26, 3l6ipiwk, ouvq2...


In [46]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'hybrid_cross_encoder_topk')
print(f"Results on the hybrid reranked set: {results_reranked}")

Results on the hybrid reranked set: {1: np.float64(0.5657142857142857), 5: np.float64(0.6185119047619048), 10: np.float64(0.6185119047619048)}


Unfortunately, we can not see any further improvements

# 4.3) Exporting hybrid approach on dev dataframe for submission on Codalab

In [47]:
df_query_dev['preds'] = df_query_dev['hybrid_topk'].apply(lambda x: x[:5])

In [48]:
df_query_dev[['post_id', 'preds']].to_csv('predictions.tsv', index=None, sep='\t')

In [49]:
df_query_dev.head()

,post_id,tweet_text,cord_uid,hybrid_cross_encoder_topk,cross_encoder_topk,hybrid_topk,bm25_topk,in_topx,preds
0,16,covid recovery: this study from the usa reveal...,3qvh482o,"[nksd3wuw, o4vvlmr4, jrqlhjsm, 8t2tic9n, hg3xp...","[o4vvlmr4, hg3xpej0, 25aj8rj5, 59up4v56, bqn29...","[25aj8rj5, styavbvi, jrqlhjsm, hg3xpej0, o4vvl...","[25aj8rj5, gatxuwz7, 59up4v56, styavbvi, 6sy80...",0.0,"[25aj8rj5, styavbvi, jrqlhjsm, hg3xpej0, o4vvl..."
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,"[r58aohnu, wk61uyrt, icgsbelo, tu1vevx9, kiq6x...","[r58aohnu, tu1vevx9, yrowv62k, mcmqx7og, zd8c1...","[r58aohnu, yrowv62k, d06npvro, mcmqx7og, mz1bo...","[r58aohnu, p0kg6dyz, 9dlaaye8, iu1d9i57, d06np...",1.0,"[r58aohnu, yrowv62k, d06npvro, mcmqx7og, mz1bo..."
2,73,I recall early on reading that researchers who...,sts48u9i,"[u5nxm9tu, 4aps0kvp, osct0lj6, o47v5vgw, sts48...","[osct0lj6, o47v5vgw, sts48u9i, gruir7aw, o877u...","[sts48u9i, 8m59whf9, gruir7aw, qkg8fwbp, osct0...","[tz2shoso, o877uul1, m1sf159a, sgo76prc, gruir...",0.2,"[sts48u9i, 8m59whf9, gruir7aw, qkg8fwbp, osct0..."
3,93,You know you're credible when NIH website has ...,3sr2exq9,"[3sr2exq9, k0f4cwig, sv48gjkk, kc3nb2pc, wbw7g...","[kdegnr6i, 3sr2exq9, k0f4cwig, sv48gjkk, z795y...","[3sr2exq9, k0f4cwig, z795y51f, sv48gjkk, 4fzzo...","[3sr2exq9, hgpiig0g, sv48gjkk, k0f4cwig, ihgxt...",1.0,"[3sr2exq9, k0f4cwig, z795y51f, sv48gjkk, 4fzzo..."
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,"[ybwwmyqy, vabb2f26, 3l6ipiwk, ouvq2wpq, sxx3y...","[ybwwmyqy, vabb2f26, ouvq2wpq, sxx3yid9, lzddn...","[ybwwmyqy, lzddnb8j, vabb2f26, ouvq2wpq, sxx3y...","[lzddnb8j, ouvq2wpq, sxx3yid9, vabb2f26, y9fqa...",1.0,"[ybwwmyqy, lzddnb8j, vabb2f26, ouvq2wpq, sxx3y..."


In [50]:
df_query_train.head()

,post_id,tweet_text,cord_uid,hybrid_topk,bm25_topk,in_topx
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5,None,"[htlvpvz5, h7hj64q5, rwgqkow3, dbgtslc8, am11y...",1.000000
1,1,this study isn't receiving sufficient attentio...,4kfl29ul,None,"[2cwvga0k, 33znyrn8, st3fyb64, 74dw6emg, b68c8...",0.000000
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8,None,"[8hkxbxz9, w98847ai, jtwb17u8, iy1enazk, pg0l9...",0.333333
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1,None,"[32ua8wb6, zxe95qy9, 30pl5tx3, l4y7v729, b97ac...",0.000000
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69,None,"[tiqksd69, k7smwz6w, b0dzhsrh, yc7cvbii, iiafu...",1.000000


## 4.4) Applying hybrid approach on test dataframe

In [59]:
df_query_test = pd.read_csv(PATH_QUERY_TEST_DATA, sep = '\t')

In [60]:
df_query_test.head()

,post_id,tweet_text
0,1,A recent research study published yesterday cl...
1,2,"""We should track the long-term effects of thes..."
2,3,"the agony of ""long haul"" covid-19 symptoms."
3,4,Home and online monitoring and assessment of b...
4,5,"it may be a long one, folks! to avoid exceedin..."


In [62]:
df_query_test['bm25_topk'] = df_query_test['tweet_text'].progress_apply(lambda x: get_top_cord_uids(x))

100%|██████████| 1446/1446 [01:05<00:00, 22.04it/s]


In [63]:
df_query_test["hybrid_topk"] = None

# Apply hybrid retrieval to each row
for idx, row in tqdm(df_query_test.iterrows(), total=len(df_query_test)):
    query_text = row["tweet_text"]
    top_docs = hybrid_retrieve(query_text, bm25, corpus_embeddings, cord_uids, top_k=20)
    df_query_test.at[idx, "hybrid_topk"] = top_docs

100%|██████████| 1446/1446 [01:28<00:00, 16.31it/s]


In [64]:
df_query_test['preds'] = df_query_test['bm25_topk'].apply(lambda x: x[:5])
df_query_test[['post_id', 'preds']].to_csv('predictions_test.tsv', index=None, sep='\t')

## 5) Several least successful approaches for documentation

First, we consider a re-ranking approach with top_k=10 instead of top_k=5 and hope that this second re-ranking step optimizes the hybrid_topk results

In [ ]:
# Initialize Cross-Encoder once
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L2-v2', max_length=512)

def rank_chunks_with_cross_encoder(query, retrieved_chunks, top_k=10):
    """
    Rank the retrieved chunks based on their relevance to the query using a Cross-Encoder.
    """
    query_chunk_pairs = [[query, chunk] for chunk in retrieved_chunks]
    scores = cross_encoder.predict(query_chunk_pairs)
    ranked_chunks = sorted(zip(retrieved_chunks, scores), key=lambda x: x[1], reverse=True)
    return ranked_chunks[:top_k]

# Initialize new column
df_query_dev["hybrid_minilm_cross_encoder_topk"] = None

# Loop over the rows to fill in reranked results
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    topk_ids = row["hybrid_topk"]

    chunk_to_uid = {}
    for uid in topk_ids:
        paper_row = df_collection.loc[df_collection["cord_uid"] == uid]
        if paper_row.empty:
            continue

        title = paper_row["title"].values[0] or ""
        abstract = paper_row["abstract"].values[0] or ""
        full_text = f"{title.strip()}\n{abstract.strip()}"
        chunk_to_uid[full_text] = uid

    reranked_chunks = rank_chunks_with_cross_encoder(query_text, list(chunk_to_uid.keys()))
    reranked_uids = [chunk_to_uid[chunk] for chunk, _ in reranked_chunks]

    # Assign directly to the dataframe
    df_query_dev.at[idx, "hybrid_minilm_cross_encoder_topk"] = reranked_uids

# Preview result
print(df_query_dev[["cord_uid", "hybrid_minilm_cross_encoder_topk"]].head())

100%|██████████| 1400/1400 [16:26<00:00,  1.42it/s]

   cord_uid                   hybrid_minilm_cross_encoder_topk
0  3qvh482o  [59up4v56, o4vvlmr4, mb18fj8a, hg3xpej0, jwei2...
1  r58aohnu  [r58aohnu, icgsbelo, h3tor88n, wk61uyrt, eay6q...
2  sts48u9i  [osct0lj6, tz2shoso, 3xw4qjoy, azl5dj74, gruir...
3  3sr2exq9  [3sr2exq9, k0f4cwig, sv48gjkk, 9mdf927z, 35xql...
4  ybwwmyqy  [ouvq2wpq, rs3umc1x, lzddnb8j, gzhc0x44, vabb2...


In [71]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'hybrid_minilm_cross_encoder_topk')
print(f"Results on the hybrid reranked set: {results_reranked}")

Results on the hybrid reranked set: {1: np.float64(0.4785714285714286), 5: np.float64(0.5498333333333333), 10: np.float64(0.5584163832199547)}


Due to no success, we want to find out if we improve the results by ranking the first retrieval only with our Bi-Encoder SentenceTransformer("all-MiniLM-L6-v2") model  

In [ ]:
def encoder_retrieve(query, corpus_embeddings, cord_uids, top_k=20):
    # Step 1: Get dense (bi-encoder) similarity scores
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    dense_scores = util.cos_sim(query_embedding, corpus_embeddings)[0].cpu().numpy()  # shape: (N,)

    # Step 2: Get top-k documents based on hybrid score
    top_indices = np.argsort(-dense_scores)[:top_k]
    top_docs = [cord_uids[i] for i in top_indices]

    return top_docs


In [73]:
df_query_dev["dense_topk"] = None

# Apply hybrid retrieval to each row
for idx, row in tqdm(df_query_dev.iterrows(), total=len(df_query_dev)):
    query_text = row["tweet_text"]
    top_docs = encoder_retrieve(query_text, corpus_embeddings, cord_uids, top_k=20)
    df_query_dev.at[idx, "dense_topk"] = top_docs

100%|██████████| 1400/1400 [00:28<00:00, 49.91it/s]


In [74]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'dense_topk')
print(f"Results on the dense set: {results_reranked}")

Results on the dense set: {1: np.float64(0.4157142857142857), 5: np.float64(0.48966666666666664), 10: np.float64(0.4993826530612245)}


For now, we can observe that the hybrid retrieval approach outperforms all other approaches